In [2]:
from keras.models import load_model
facenet_base = load_model('facenet.h5')

C:\Users\adishjain01\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [9]:
from mtcnn.mtcnn import MTCNN
import cv2
from numpy import expand_dims
detector = MTCNN()

In [6]:
def get_face(filename, required_size=(160, 160)):
    image = cv2.imread(filename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(image)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = image[y1:y2, x1:x2]
    face = cv2.resize(face, required_size)
    return face.astype('float32')

def get_embedding(model, face):
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    samples = expand_dims(face, axis=0)
    yhat = model.predict(samples)
    return yhat[0]


In [7]:
image1_path = 'test1.jpg'
image2_path = 'test2.jpg'

In [10]:
face1 = get_face(image1_path)
face1_embd = get_embedding(facenet_base, face1)

face2 = get_face(image2_path)
face2_embd = get_embedding(facenet_base, face2)

In [12]:
import keras.backend as K

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

face_detector = load_model("best_model_saved_embd_conloss.h5", custom_objects={'contrastive_loss':contrastive_loss})
face_detector.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_36 (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
input_37 (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
model_22 (Model)                (None, 128)          17024       input_36[0][0]                   
                                                                 input_37[0][0]                   
________________________________________________________________________________________________

In [20]:
distance = face_detector.predict([[face1_embd], [face2_embd]])[0][0]

In [21]:
similarity_score = 1 - distance

In [22]:
if similarity_score > 0.5:
    print("The images are of same person")
    

The images are of same person
